# HW2 Analysis

In [1]:
# File managing imports
import os
import sys
import pickle as pkl
import json

# np and torch stuff
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import one_hot

# Imports specific to doc2vec and NLP
import gensim
from gensim.test.utils import get_tmpfile
import nltk

#sys.path.append(os.getcwd() + '/..')

# To read the files and qrels
import read_ap
import download_ap

# Model and helper imports
from model import SkipGramNet
from helper import build_dicts

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [306]:
query = """More than 150 former officers of the
overthrown South Vietnamese government have been released from a
re-education camp after 13 years of detention, the official Vietnam
News Agency reported Saturday.
The report from Hanoi, monitored in Bangkok, did not give
specific figures, but said those freed Friday included an
ex-Cabinet minister, a deputy minister, 10 generals, 115
field-grade officers and 25 chaplains.
   It quoted Col. Luu Van Ham, director of the Nam Ha camp south of
Hanoi, as saying all 700 former South Vietnamese officials who had
been held at the camp now have been released.
   They were among 1,014 South Vietnamese who were to be released
from re-education camps under an amnesty announced by the Communist
government to mark Tet, the lunar new year that begins Feb. 17.
   The Vietnam News Agency report said many foreign journalists and
a delegation from the Australia-Vietnam Friendship Association
attended the Nam Ha release ceremony.
   It said Lt. Gen. Nguyen Vinh Nghi, former commander of South
Vietnam's Third Army Corps, and Col. Tran Duc Minh, former director
of the Army Infantry Officers School, expressed ``gratitude to the
government for its humane treatment in spite of the fact that most
of them (detainees) had committed heinous crimes against the
country and people.''
   The prisoners had been held without formal charges or trial
since North Vietnam defeated the U.S.-backed South Vietnamese
government in April 1975, ending the Vietnam War.
   Communist authorities had called the prisoners war criminals and
said they had to learn how to become citizens of the new society.
   Small numbers had been released occasionally without publicity
but the government announced last year that 480 political prisoners
would be freed to mark National Day on Sept. 2.
   On Thursday, Vice Minister of Information Phan Quang said 1,014
would be released under the Tet amnesty.
   He reported a total of 150 prisoners would remain in the camps,
which he said once held 100,000.
   ``Depending on their repentance, they will gradually be released
within a short period of time,'' Quang said.
   He said many of the former inmates would return to their
families in Ho Chi Minh City, formerly the South Vietnamese capital
of Saigon.
   The amnesties apparently are part of efforts by Communist Party
chief Nguyen Van Linh to heal internal divisions and improve
Vietnam's image abroad."""

In [60]:
query = "The president of the United States threatened with sanctions against Iran"

In [61]:
d2vmodel = gensim.models.doc2vec.Doc2Vec.load("d2v.model")

In [62]:
test_data = read_ap.process_text(query)

In [40]:
v1 = d2vmodel.infer_vector(test_data)
#print("V1 infer: ", v1)

In [41]:
#model.docvecs.most_similar([v1])

In [42]:
def ranking(query, model, num_results = 10):
    query = read_ap.process_text(query)
    vec = model.infer_vector(query)
    
    return model.docvecs.most_similar([vec], topn = num_results)

In [43]:
#ranks = ranking(query, model = d2vmodel, num_results = 164557)
ranks = ranking(query, model = d2vmodel, num_results = 10)

In [44]:
ranks

[('AP890807-0056', 0.7527885437011719),
 ('AP880720-0091', 0.7495495676994324),
 ('AP890904-0085', 0.7414106726646423),
 ('AP880728-0091', 0.7402305006980896),
 ('AP891211-0071', 0.7330125570297241),
 ('AP890618-0047', 0.7291489243507385),
 ('AP880731-0005', 0.724372923374176),
 ('AP890117-0091', 0.7233673334121704),
 ('AP880228-0003', 0.7211905121803284),
 ('AP880418-0049', 0.7195896506309509)]

In [248]:
# Dictionary containing index to specific document
i2str = {}

for i, doc in enumerate(docs_by_id):
    i2str[i] = doc

# Train New Doc2Vec

In [2]:
# Make sure we have the dataset
download_ap.download_dataset()

# pre-process the text
docs_by_id = read_ap.get_processed_docs()
# Instead of preprocessing maybe just do lemmatizing?

# get qrels
qrels, queries = read_ap.read_qrels()

Docs already processed. Loading from disk


In [14]:
def read_corpus(docs_by_id, tokens_only=False):
    
    for key, doc in docs_by_id.items():
        if tokens_only:
            yield doc
        else:
            # If data is used for training we require tags
            yield gensim.models.doc2vec.TaggedDocument(doc, [key])

In [15]:
train_corpus = list(read_corpus(docs_by_id))
test_corpus  = list(read_corpus(docs_by_id, tokens_only = True))

In [16]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=200, min_count=250, epochs=15, window = 5)

In [ ]:
model.build_vocab(train_corpus)

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save("d2v_200_10_5-2.model")

# Retrieval and Evaluation

# Latent Dirichlet Allocation

In [56]:
# Make sure we have the dataset
download_ap.download_dataset()

# pre-process the text
docs_by_id = read_ap.get_processed_docs()
# Instead of preprocessing maybe just do lemmatizing?

# get qrels
qrels, queries = read_ap.read_qrels()

Docs already processed. Loading from disk


In [15]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
docs = [doc for key, doc in docs_by_id.items()]

# bigram = Phrases(docs, min_count=20)

# for key, doc in docs_by_id.items():
#     for token in bigram[docs_by_id[key]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             docs_by_id[key].append(token)
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [27]:
# Save bigram dictionary
jsondocs = json.dumps(docs)
f = open("bigram.json","w")
f.write(jsondocs)
f.close()

In [67]:
if os.path.isfile("bigram.json"):
    docs = json.load(open("bigram.json","r"))

In [68]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [69]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [70]:
# LDA model import
from gensim.models import LdaModel

In [107]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 1
iterations = 1
eval_every = None  # Don't evaluate model perplexity, takes too much time.

In [217]:
# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
word2id = dictionary.token2id

In [109]:
# Train LDA model
model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [49]:
# Save model
model.save("lda.model")

In [149]:
# Load a pretrained model
model = LdaModel.load("lda.model")

In [189]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.5004.
[([(0.023758994, 'bush'),
   (0.015403773, 'hous'),
   (0.015127242, 'presid'),
   (0.008998448, 'senat'),
   (0.008307407, 'democrat'),
   (0.007755701, 'vote'),
   (0.0066285995, 'congress'),
   (0.0065940414, "n't"),
   (0.0064757434, 'state'),
   (0.0063961814, 'committe'),
   (0.0060074544, 'administr'),
   (0.005851559, 'white'),
   (0.0054646637, 'bill'),
   (0.005427055, 'republican'),
   (0.0048162234, 'white_hous'),
   (0.0047033317, 'nation'),
   (0.004427831, 'elect'),
   (0.004368095, 'go'),
   (0.0043071145, 'support'),
   (0.004256229, 'campaign')],
  -1.2514183593588153),
 ([(0.014035112, 'million'),
   (0.01178711, 'compani'),
   (0.007436369, 'billion'),
   (0.006537955, 'percent'),
   (0.006073426, 'new'),
   (0.0045764567, 'program'),
   (0.0044825953, 'plan'),
   (0.0041710585, 'feder'),
   (0.0041421424, 'state'),
   (0.0040388815, 'industri'),
   (0.0037687365, 'busi'),
   (0.0036146569, 'also'),
   (0.0035998167, 'use'),
   (0.0

# Kullback-Leibler Divergence

In [78]:
from gensim.matutils import kullback_leibler

In [150]:
for doc in docs:
    print(doc[:10])
    break

['150', 'former', 'offic', 'overthrown', 'south', 'vietnames', 'govern', 'releas', 're-educ', 'camp']


In [221]:
query = "150 former offic overthrown south vietnames govern releas re-educ camp poep"

In [223]:
query = read_ap.process_text(query)
query

['150',
 'former',
 'offic',
 'overthrown',
 'south',
 'vietnam',
 'govern',
 'relea',
 're-educ',
 'camp',
 'poep']

In [225]:
# query = model.id2word.doc2bow(query)
# query
query = dictionary.doc2bow(query)
query

[(4, 1),
 (28, 1),
 (68, 1),
 (76, 1),
 (120, 1),
 (122, 1),
 (133, 1),
 (150, 1),
 (164, 1)]

In [266]:
query = """More than 150 former officers of the
overthrown South Vietnamese government have been released from a
re-education camp after 13 years of detention, the official Vietnam
News Agency reported Saturday.
The report from Hanoi, monitored in Bangkok, did not give
specific figures, but said those freed Friday included an
ex-Cabinet minister, a deputy minister, 10 generals, 115
field-grade officers and 25 chaplains.
   It quoted Col. Luu Van Ham, director of the Nam Ha camp south of
Hanoi, as saying all 700 former South Vietnamese officials who had
been held at the camp now have been released.
   They were among 1,014 South Vietnamese who were to be released
from re-education camps under an amnesty announced by the Communist
government to mark Tet, the lunar new year that begins Feb. 17.
   The Vietnam News Agency report said many foreign journalists and
a delegation from the Australia-Vietnam Friendship Association
attended the Nam Ha release ceremony.
   It said Lt. Gen. Nguyen Vinh Nghi, former commander of South
Vietnam's Third Army Corps, and Col. Tran Duc Minh, former director
of the Army Infantry Officers School, expressed ``gratitude to the
government for its humane treatment in spite of the fact that most
of them (detainees) had committed heinous crimes against the
country and people.''
   The prisoners had been held without formal charges or trial
since North Vietnam defeated the U.S.-backed South Vietnamese
government in April 1975, ending the Vietnam War.
   Communist authorities had called the prisoners war criminals and
said they had to learn how to become citizens of the new society.
   Small numbers had been released occasionally without publicity
but the government announced last year that 480 political prisoners
would be freed to mark National Day on Sept. 2.
   On Thursday, Vice Minister of Information Phan Quang said 1,014
would be released under the Tet amnesty.
   He reported a total of 150 prisoners would remain in the camps,
which he said once held 100,000.
   ``Depending on their repentance, they will gradually be released
within a short period of time,'' Quang said.
   He said many of the former inmates would return to their
families in Ho Chi Minh City, formerly the South Vietnamese capital
of Saigon.
   The amnesties apparently are part of efforts by Communist Party
chief Nguyen Van Linh to heal internal divisions and improve
Vietnam's image abroad."""

In [267]:
query = read_ap.process_text(query)

In [268]:
query = dictionary.doc2bow(query)
query

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 3),
 (14, 1),
 (15, 2),
 (16, 1),
 (17, 1),
 (19, 2),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 5),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 2),
 (38, 1),
 (39, 1),
 (40, 3),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 2),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (64, 1),
 (65, 1),
 (67, 1),
 (68, 5),
 (69, 1),
 (70, 2),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 5),
 (77, 1),
 (78, 1),
 (79, 2),
 (80, 1),
 (81, 2),
 (82, 1),
 (83, 1),
 (85, 3),
 (86, 1),
 (88, 1),
 (89, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (100, 1),
 (101, 1),
 (102, 1),
 (104, 1),
 (105, 2),
 (106, 2),
 (107, 2),
 (109, 3),
 (110, 1),
 (112, 2),


In [269]:
print(model[query])

[(1, 0.047774028), (2, 0.7972291), (4, 0.09066219), (6, 0.019111032), (8, 0.042699546)]


In [289]:
print(model[corpus[0]])

[(1, 0.047015667), (2, 0.80070025), (4, 0.0868585), (8, 0.06223184)]


In [290]:
query = model[query]

[(1, 0.04778037),
 (2, 0.79723895),
 (4, 0.09066447),
 (6, 0.019084737),
 (8, 0.042707335)]

In [291]:
new_query = gensim.matutils.sparse2full(model[query], num_topics)

In [292]:
new_corp0 = gensim.matutils.sparse2full(model[corpus[0]], num_topics)

In [293]:
new_corp1 = gensim.matutils.sparse2full(model[corpus[1]], num_topics)

In [294]:
kullback_leibler(new_query, new_corp1)

inf

In [295]:
new_corp0

array([0.        , 0.04702356, 0.80069256, 0.        , 0.08685537,
       0.        , 0.        , 0.        , 0.06223481, 0.        ],
      dtype=float32)

In [296]:
new_query

array([0.0365147 , 0.12522797, 0.0579267 , 0.09677395, 0.06265194,
       0.03005568, 0.47769132, 0.02692753, 0.04017168, 0.04605851],
      dtype=float32)

# Function that ranks documents given a query

In [297]:
def ranking_LDA(query, model, num_results = 10, num_topics = num_topics):
    scores = []
    
    for i in range(len(corpus)):
        doc = gensim.matutils.sparse2full(model[corpus[i]], num_topics)
        neg_kl = float(-1 * kullback_leibler(query, doc))
        scores.append((i2str[i], neg_kl))

    return scores

In [298]:
scores = ranking_LDA(new_query, model)

In [299]:
sort = sorted(scores, key=lambda x: x[1], reverse = True)